# Preprocessing of the dataset

The knowledge created on this dataset will be exported as transformers and saved on **libs/transformers.py** so it can be reusable in sklearn pipelines 

In [420]:
import sys
import os
sys.path.append(os.getcwd())

import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold

In [421]:
df = pd.read_csv('../beijing.csv', encoding="gbk")

/tmp/ipykernel_375354/809342345.py:1: DtypeWarning: Columns (1,11,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../beijing.csv', encoding="gbk")


# Deleting assured columns

In [422]:
df.drop(['url'], axis=1, inplace=True)

# Mutating or one-hot-encoding columns since string stuff 

In [423]:
# convert tradeTime date column to timestamp
df['tradeTime'] = pd.to_datetime(df['tradeTime']).astype(int)/ 10**9

In [424]:
# transforming id column to numeric
df['id'] = df.index + 1

In [425]:
# This column unfortunately has the same labels as string and int object.
# There is a label '#NAME?' which is not clear for what use it is. Since only 32 of them, we can fill it with the median if the column (Since discrete values)
df.groupby('livingRoom')['id'].nunique()

livingRoom
0            11
1         42138
2         77256
3         31567
4          3349
5           537
6           126
7            22
8             3
9             2
#NAME?       32
0            17
1         40248
2         83333
3         36044
4          3472
5           570
6           102
7            15
8             4
9             3
Name: id, dtype: int64

In [426]:
def mapper(x):
    if x == '#NAME?':
        return np.nan
    else: return int(x)
df['livingRoom'] = df['livingRoom'].apply(mapper)
df['livingRoom'].fillna(df['livingRoom'].median(), inplace=True)
df['livingRoom'] = df['livingRoom'].astype(int)
df.groupby('livingRoom')['id'].nunique()

livingRoom
0        28
1     82386
2    160621
3     67611
4      6821
5      1107
6       228
7        37
8         7
9         5
Name: id, dtype: int64

In [427]:
chars = ['中', '低', '底', '顶', '高']
def mapper(x):
    for c in chars:
        if c in str(x):
            return np.nan
    return int(x)
df['drawingRoom'] = df['drawingRoom'].apply(mapper)
df['drawingRoom'].fillna(df['drawingRoom'].median(), inplace=True)
df['drawingRoom'] = df['drawingRoom'].astype(int)
df.groupby('drawingRoom')['id'].nunique()

drawingRoom
0     19686
1    225691
2     72502
3       918
4        47
5         7
Name: id, dtype: int64

In [428]:
chars = ['未知']
def mapper(x):
    for c in chars:
        if c in str(x):
            return np.nan
    return int(x)
df['bathRoom'] = df['bathRoom'].apply(mapper)
df['bathRoom'].fillna(df['bathRoom'].median(), inplace=True)
df['bathRoom'] = df['bathRoom'].astype(int)
df.groupby('bathRoom')['id'].nunique()

bathRoom
0          915
1       261490
2        52606
3         3240
4          489
5           69
6            9
7            3
1990         2
1994         1
1996         1
2000         3
2003         7
2004         4
2005         2
2006         9
2011         1
Name: id, dtype: int64

In [429]:
df['floorType'] = df['floor'].copy()
def mapper(x):
    splitted = x.split(' ')
    finalStr = x
    if len(splitted) == 2:
        finalStr = splitted[0]
    translations = {
        '中': 'middle',
        '低': 'low',
        '底': 'bottom',
        '未知': np.nan,
        '混合结构': 'hybrid',
        '钢混结构': 'steel',
        '顶': 'top',
        '高': 'high'
    }
    return translations[finalStr]
    
df['floorType'] = df['floorType'].apply(mapper)
df.groupby('floorType')['id'].nunique()

floorType
bottom     26080
high       70094
hybrid        15
low        64136
middle    119832
steel         17
top        37406
Name: id, dtype: int64

In [430]:
def mapper(x):
    splitted = x.split(' ')
    if len(splitted) == 2:
        return splitted[1]
    else:
        return np.nan
df['floor'] = df['floor'].apply(mapper)
df['floor'].fillna(df['floor'].median(), inplace=True)
df['floor'] = df['floor'].astype(int)

In [431]:
df.groupby('floor')['id'].nunique()

floor
1        332
2        185
3       1098
4       3023
5      13467
6     107791
7       9961
8       2342
9       7109
10      5707
11      9675
12      9118
13      4311
14      8837
15      7283
16     15999
17      5233
18     23013
19      4721
20     10025
21      9018
22      9228
23      3539
24     13594
25      6403
26      6886
27      4854
28      7449
29      1836
30      2472
31       670
32      2477
33       571
34       555
35         4
36        12
37        16
42        34
57         2
63         1
Name: id, dtype: int64

In [432]:
df.groupby('buildingType')['id'].nunique()
df = df[df['buildingType'] > 1]  
df['buildingType'] = df['buildingType'].astype(int)
df.groupby('buildingType')['id'].nunique()

buildingType
2       137
3     59715
4    172405
Name: id, dtype: int64

In [433]:
df.groupby('constructionTime')['id'].nunique()
chars = ['未知']
def mapper(x):
    for c in chars:
        if c in str(x):
            return np.nan
    return int(x)
df['constructionTime'] = df['constructionTime'].apply(mapper)
df['constructionTime'].fillna(df['constructionTime'].median(), inplace=True)
df['constructionTime'] = df['constructionTime'].astype(int)

In [434]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 232257 entries, 2 to 318850
Data columns (total 26 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   232257 non-null  int64  
 1   Lng                  232257 non-null  float64
 2   Lat                  232257 non-null  float64
 3   Cid                  232257 non-null  int64  
 4   tradeTime            232257 non-null  float64
 5   DOM                  117692 non-null  float64
 6   followers            232257 non-null  int64  
 7   totalPrice           232257 non-null  float64
 8   price                232257 non-null  int64  
 9   square               232257 non-null  float64
 10  livingRoom           232257 non-null  int64  
 11  drawingRoom          232257 non-null  int64  
 12  kitchen              232257 non-null  int64  
 13  bathRoom             232257 non-null  int64  
 14  floor                232257 non-null  int64  
 15  buildingType     

## Missing values

In [435]:
missing=df.isnull().mean()>0 
missing[missing == True]

DOM                 True
communityAverage    True
floorType           True
dtype: bool

In [436]:
# dependant variables: DOM, buildingType, elevator, fiveYearsProperty, subway, communityAverage
# target variable: totalPrice
cols_to_use = ['DOM', 'buildingType', 'elevator', 'fiveYearsProperty', 'subway', 'communityAverage', 'totalPrice']
data_simp = df[cols_to_use]
data_simp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 232257 entries, 2 to 318850
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   DOM                117692 non-null  float64
 1   buildingType       232257 non-null  int64  
 2   elevator           232257 non-null  float64
 3   fiveYearsProperty  232257 non-null  float64
 4   subway             232257 non-null  float64
 5   communityAverage   231963 non-null  float64
 6   totalPrice         232257 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 14.2 MB


In [437]:
data_simp.isnull().mean()

DOM                  0.493268
buildingType         0.000000
elevator             0.000000
fiveYearsProperty    0.000000
subway               0.000000
communityAverage     0.001266
totalPrice           0.000000
dtype: float64

In [438]:
df['DOM'].fillna(df['DOM'].median(), inplace=True)

# Can i accept 0 as a value for these missing data?

From the six variables, only one have a missinge percentage greater than 5%, the DOM variable
- The DOM variable means **Active days on market** and it has a missing percentage of almost 50% but the .isnull function from pandas recognizes the value zero as null but zero may be an important label. It could mean that the property on sale could be zero active days on market. We need to find out if it is completely random or not. If it is not random, than probably some zeros are treated the same as null and as zero active days on market.

Hypothesen:
 - H(null): The data was produced in a random manner.  
 - H(alternative): The data was not produced in a random manner.

In [439]:
from statsmodels.sandbox.stats.runs import runstest_1samp

dom_values = df['DOM'].to_list()
print("mean: ", df['DOM'].mean())

# Perform Runs test
runstest_1samp(dom_values, cutoff=df['DOM'].mean())

mean:  17.89306673211141


(-305.7147323550679, 0.0)

z_stat: -313.44; p-value: 0%  
Since the p-value is smaller than 5%, we need to accept the null hypothesis. 
This means, that the zero values of DOM are randomly generated. Let's analyse further

### Check for near-zero variance and eliminate columns with low variance

In [440]:
# Get variance of each column
df.var()

/tmp/ipykernel_375354/2452669276.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.var()


id                     8.531541e+09
Lng                    1.376536e-02
Lat                    9.654671e-03
Cid                    2.212686e+24
tradeTime              2.680363e+15
DOM                    1.347836e+03
followers              1.182295e+03
totalPrice             5.134979e+04
price                  4.900779e+08
square                 1.292596e+03
livingRoom             5.834574e-01
drawingRoom            2.619567e-01
kitchen                1.034109e-02
bathRoom               1.901105e-01
floor                  4.100881e+01
buildingType           1.927555e-01
constructionTime       8.268303e+01
renovationCondition    1.707553e+00
buildingStructure      3.757172e+00
ladderRatio            8.627309e+08
elevator               2.454697e-01
fiveYearsProperty      2.304293e-01
subway                 2.460557e-01
district               8.148298e+00
communityAverage       5.383763e+08
dtype: float64

In [441]:
selector = VarianceThreshold(threshold=0.05) # 0.05 is 5% of the variance of the column^
df_filtered = selector.fit_transform(df) # fit and transform the data

ValueError: could not convert string to float: 'middle'

In [ ]:
df_filtered

AttributeError: 'VarianceThreshold' object has no attribute 'var'